## **Fase 1 - Extract**
*En esta fase inicial, el enfoque está en la extracción de datos brutos directamente desde la plataforma de Airbnb. Utilizando un scraper automatizado, extraemos información esencial sobre las propiedades disponibles, como ubicación, precios, tipo de alojamiento, disponibilidad y reseñas de los usuarios. La extracción es un paso crucial para asegurar que todos los datos relevantes se reúnan y estén listos para su análisis. Nos aseguramos de que los datos se extraigan de forma precisa y consistente, lo cual es fundamental para el éxito de las fases siguientes del proceso. El objetivo de esta fase es consolidar la información extraída en un formato accesible (DataFrame) de manera eficiente y automatizada, para que posteriormente puedan ser limpiados y transformados.* **(necesita edicion - version test)**

In [8]:
# ETL imports
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep

# General 
import pandas as pd
import numpy as np
import json          
import os
from tqdm import tqdm
        

In [5]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

In [6]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts)

In [9]:
# Lista de distritos de Barcelona
distritos = [
    "Ciutat Vella",
    "Eixample",
    "Sants-Montjuïc",
    "Les Corts",
    "Sarrià-Sant Gervasi",
    "Gràcia",
    "Horta-Guinardó",
    "Nou Barris",
    "Sant Andreu",
    "Sant Martí"
]

# Creamos una lista para almacenar enlaces de distritos
link_distrito = []

# URL base con el formato de Airbnb
url_base = "https://www.airbnb.es/s/{distrito}--Barcelona--España/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&monthly_start_date=2024-11-01&monthly_length=3&monthly_end_date=2025-02-01&price_filter_input_type=0&channel=EXPLORE&query={distrito}%2C%20Barcelona%2C%20España&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click"

# Iterar sobre cada distrito y generar el enlace
for distrito in distritos:
    # Reemplazar espacios por %20 y caracteres especiales
    distrito_url = distrito.replace(" ", "%20").replace("à", "%C3%A0").replace("ç", "%C3%A7").replace("í", "%C3%AD")
    
    # Formatear el enlace con el nombre del distrito
    enlace = url_base.format(distrito=distrito_url)

    link_distrito.append(enlace)

# Lista vacía para almacenar los links de habitaciones
links_to_scrapp = []

# Accedemos a las paginas de cada distrito
for distrito in tqdm(link_distrito, desc="Accediendo a distritos"):
    
    driver.get(distrito)

    # Pausa
    sleep(3)

    # Bucle para la paginación y extración
    while len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
        try:
            sleep(2)
            # Obtener el código fuente de la página actual
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraer todos los enlaces de habitaciones en la página actual
            link_habitacion = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/rooms/"]')

            # Lista de urls target
            links_target = []

            # Recorrer todos los enlaces y filtrar los que contienen '/rooms'
            for _ in link_habitacion:
                link = _.get_attribute('href')
                if link and '/rooms/' in link:
                    links_target.append(link)

            # Eliminar duplicados en la lista de enlaces
            links_filtered = set(links_target)
            print(f"Número de links filtrados en la página: {len(links_filtered)}")

            # Añadimos los links en la lista de scrappeo
            for url in links_filtered:
                if len(links_to_scrapp) < 500:  # Limitar a 500 enlaces
                    links_to_scrapp.append(url)
                    print(f'Agregando URL: {url}')
                else:
                    break

            print(f"Total de links extraídos hasta ahora: {len(links_to_scrapp)}")

            if len(links_to_scrapp) >= 500:  
                print('Total de links atingido. Parando a extração...')
                break

            # Pausa antes de hacer click en la próxima página
            sleep(2)

            # Buscar y hacer click en el botón "Siguiente"
            next_button = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Siguiente"]')
            next_button.click()
            print('Siguiente página...')

        except Exception as e:
            print('No se pudo cargar la siguiente página o no hay más páginas.')
            print(e)
            break

# Imprimir a lista final de enlaces
print("Extracción completa. Total de links:", len(links_to_scrapp))

Accediendo a distritos:   0%|          | 0/10 [00:00<?, ?it/s]

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/14006090?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-17&check_out=2024-11-22&source_impression_id=p3_1730030821_P3AwJzO8Hv5GvnuZ&previous_page_section_name=1000&federated_search_id=8a6ca1cd-aebd-4e34-b8f0-a916bd99661b
Agregando URL: https://www.airbnb.es/rooms/852372822813054452?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1667248347&search_mode=regular_search&check_in=2024-11-28&check_out=2024-12-03&source_impression_id=p3_1730030821_P3eLGFkbaKs3MXtD&previous_page_section_name=1000&federated_search_id=8a6ca1cd-aebd-4e34-b8f0-a916bd99661b
Agregando URL: https://www.airbnb.es/rooms/1248794356730472452?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-01&check_out=2024-11-06&source_impression_id=p3_1730030821_P3_nPpVpMlehAVrp&pre

Accediendo a distritos:  10%|█         | 1/10 [01:41<15:15, 101.68s/it]

No se pudo cargar la siguiente página o no hay más páginas.
Message: no such element: Unable to locate element: {"method":"css selector","selector":"a[aria-label="Siguiente"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x009C3853+24035]
	(No symbol) [0x0094BBE4]
	(No symbol) [0x0082C2D3]
	(No symbol) [0x0086DC86]
	(No symbol) [0x0086DECB]
	(No symbol) [0x008AB9D2]
	(No symbol) [0x0088FED4]
	(No symbol) [0x008A953F]
	(No symbol) [0x0088FC26]
	(No symbol) [0x0086218C]
	(No symbol) [0x0086310D]
	GetHandleVerifier [0x00C69683+2800659]
	GetHandleVerifier [0x00CC423E+3172302]
	GetHandleVerifier [0x00CBCE52+3142626]
	GetHandleVerifier [0x00A66C00+692624]
	(No symbol) [0x00954BFD]
	(No symbol) [0x00951908]
	(No symbol) [0x00951AA0]
	(No symbol) [0x00943F50]
	BaseThreadInitThunk [0x76A67BA9+25]
	RtlInitializeExceptionChain

Accediendo a distritos:  20%|██        | 2/10 [03:06<12:13, 91.75s/it] 

Número de links filtrados en la página: 18
Agregando URL: https://www.airbnb.es/rooms/20061032?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-12-17&check_out=2024-12-22&source_impression_id=p3_1730030997_P36CkzOV6q8AczK1&previous_page_section_name=1000&federated_search_id=25c4060d-f364-4ada-a488-4d1340506656
Agregando URL: https://www.airbnb.es/rooms/1275279546374617011?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-04&check_out=2024-11-09&source_impression_id=p3_1730030997_P3nJttz4pp21WAU_&previous_page_section_name=1000&federated_search_id=25c4060d-f364-4ada-a488-4d1340506656
Agregando URL: https://www.airbnb.es/rooms/7953354?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-12-08&check_out=2024-12-13&source_impression_id=p3_1730030997_P3xZ6zIUjrBix8uN&previous_page_section_name=1000&federated_search_id=25c406

Accediendo a distritos: 100%|██████████| 10/10 [03:56<00:00, 23.68s/it]

Extracción completa. Total de links: 500


In [10]:
# Función para extraer el texto de un elemento dado
def extract_text(soup, tag, class_name, index=0, default=np.nan):
    try:
        elements = soup.find_all(tag, class_=class_name)
        return elements[index].text.strip() if elements else default  # Retorna el texto del elemento especificado
    except (AttributeError, IndexError):
        return default

# Diccionario para los campos a extraer, organizados en el orden deseado
fields = {
    "Titles": ("span", "l1h825yc", 1),  # Título del alojamiento
    "Host_name": ("div", "t1pxe1a4", 0),  # Nombre del anfitrión
    "Property_types": ("h2", "hpipapi", 0),  # Tipo de propiedad
    "Prices_per_night": ("span", "_11jcbg2", 0),  # Precio por noche
    "Check_ins": ("div", "i1303y2k", 0),  # Check-in
    "Check_outs": ("div", "i1303y2k", 1),  # Check-out
    "Cleaning_fees": ("span", "_1k4xcdh", 1),  # Tasa de limpieza
    "Location": ("div", "_152qbzi", 0),  # Ubicación
}

# Limitar a 500 enlaces para la muestra
sample_links = links_to_scrapp[:100]

# Lista para almacenar los datos extraídos
data_list = []

# Iterar sobre cada enlace en la muestra
for link in tqdm(sample_links, desc="Extrayendo datos"):

    # Asegurarse de que el enlace tenga el parámetro 'adults'
    if 'adults' not in link:
        link += '&adults=1'
    else:
        link += '?adults=1'    

    driver.get(link)  # Acceder a la página
    sleep(3)  # Espera para cargar la página

    # Obtener el objeto BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    data = {'urls': link}  # Agregar la URL al diccionario de datos

    # Extraer usando el diccionario y la función de extracción
    data.update({field: extract_text(soup, *details) for field, details in fields.items()})

    # Obtener las reseñas y calificaciones
    reviews_data = driver.find_elements(By.XPATH, '//span[@aria-hidden="true"]')
    ratings_list = []
    num_reviews_list = []
    rating = num_reviews = np.nan  # Inicializa como NaN


    # Procesar las reseñas
    for review in reviews_data:
        text = review.text

        if "(" in text:  # Verificar si el formato es el correcto
            rating = text.split(' ')[0]
            num_reviews = text.split('(')[1].replace(')', '')
            ratings_list.append(rating)  # Añadir la calificación a la lista
            num_reviews_list.append(num_reviews)  # Extraer el número de reseñas y quitar los paréntesis
            break  # Salir del bucle después de encontrar el primer formato correcto

    # Añadir calificaciones y número de reseñas al diccionario de datos
    data['Ratings'] = rating
    data['Num_reviews'] = num_reviews

    # Extracción de detalles (camas, baños, etc.) usando el selector CSS
    details = driver.find_elements(By.CSS_SELECTOR, "div.o1kjrihn ol.lgx66tx")
    details_dict = {}
    for detail in details:
        detail_text = detail.text.strip()
        # Lógica para identificar qué representa cada detalle
        if "camas" in detail_text.lower():
            details_dict['Beds'] = detail_text
        elif "dormitorios" in detail_text.lower():
            details_dict['Rooms'] = detail_text
        elif "baños" in detail_text.lower():
            details_dict['Baths'] = detail_text

    # Agregar detalles al diccionario de datos
    data.update(details_dict)

    # Agregar el número máximo de huéspedes
    n_guests = driver.find_elements(By.CSS_SELECTOR, "div[data-plugin-in-point-id='POLICIES_DEFAULT'] div.i1303y2k span")
    guests_count = np.nan  # Valor predeterminado en caso de no encontrar
    for guest in n_guests:
        if 'huésped' in guest.text or 'huéspedes' in guest.text:
            guests_count = guest.text.split()[1]  # Extraer solo el número de huéspedes
            break  # Salir del bucle si se encuentra el número de huéspedes

    # Añadir el número máximo de huéspedes al diccionario de datos
    data['Maximum_guests'] = guests_count

    # Añadir los datos extraídos a la lista
    data_list.append(data)

# Crear el DataFrame final con los datos extraídos
df = pd.DataFrame(data_list)

Extrayendo datos:   1%|          | 1/100 [00:03<05:56,  3.60s/it]

Total de reseñas encontradas: 2
Texto de la reseña: 
Texto de la reseña: 
Total de reseñas encontradas: 6
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 


Extrayendo datos:   2%|▏         | 2/100 [00:07<06:25,  3.93s/it]

Total de reseñas encontradas: 47
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
T

Extrayendo datos:   3%|▎         | 3/100 [00:11<06:29,  4.01s/it]

...

Extrayendo datos:  99%|█████████▉| 99/100 [06:52<00:04,  4.30s/it]

Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Total de reseñas encontradas: 57
Texto de la reseña: 
Texto de la reseña:  · 
Texto de la reseña: 
Texto de la reseña:  · 
Texto de la reseña: 
Texto de la reseña:  · 
Texto de la reseña: ·
Texto de la reseña: 2 evaluaciones
Texto de la reseña: ·
Texto de la reseña: ·
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseña: 
Texto de la reseñ

Extrayendo datos: 100%|██████████| 100/100 [06:57<00:00,  4.29s/it]

Texto de la reseña: 
Texto de la reseña: 


Extrayendo datos: 100%|██████████| 100/100 [06:57<00:00,  4.17s/it]


In [11]:
df.head()

,urls,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Cleaning_fees,Location,Ratings,Num_reviews,Maximum_guests,Beds,Rooms
0,https://www.airbnb.es/rooms/14006090?adults=1&...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.es/rooms/852372822813054452...,Este alojamiento tiene una ubicación estratégi...,Anfitrión: Maria,"Habitación en Barcelona, España",40 €,NaN,NaN,34 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,3 camas,NaN
2,https://www.airbnb.es/rooms/124879435673047245...,Es una gran habitación doble queen con oficina...,Anfitrión: Mazen,"Alojamiento entero: apartamento en Barcelona, ...",82 €,Horario de llegada: de 15:00 a 22:00,Salida antes de las 11:00,20 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,NaN,NaN
3,https://www.airbnb.es/rooms/125429912134817995...,Este anfitrión tiene 2081 evaluaciones de otro...,Quédate con Rubén,"Habitación en Barcelona, España",60 €,Llegada a partir de las 15:00,Máximo 2 huéspedes,NaN,"Barcelona, Catalunya, España",NaN,NaN,2,2 camas · Baño compartido,NaN
4,https://www.airbnb.es/rooms/583987826544046548...,Disfruta de la sencillez de este alojamiento t...,Quédate con Lina Maria,"Habitación en Barcelona, España",25 €,Horario de llegada: de 16:00 a 21:00,Salida antes de las 12:00,5 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,NaN,NaN


In [12]:
df

,urls,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Cleaning_fees,Location,Ratings,Num_reviews,Maximum_guests,Beds,Rooms
0,https://www.airbnb.es/rooms/14006090?adults=1&...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.es/rooms/852372822813054452...,Este alojamiento tiene una ubicación estratégi...,Anfitrión: Maria,"Habitación en Barcelona, España",40 €,NaN,NaN,34 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,3 camas,NaN
2,https://www.airbnb.es/rooms/124879435673047245...,Es una gran habitación doble queen con oficina...,Anfitrión: Mazen,"Alojamiento entero: apartamento en Barcelona, ...",82 €,Horario de llegada: de 15:00 a 22:00,Salida antes de las 11:00,20 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,NaN,NaN
3,https://www.airbnb.es/rooms/125429912134817995...,Este anfitrión tiene 2081 evaluaciones de otro...,Quédate con Rubén,"Habitación en Barcelona, España",60 €,Llegada a partir de las 15:00,Máximo 2 huéspedes,NaN,"Barcelona, Catalunya, España",NaN,NaN,2,2 camas · Baño compartido,NaN
4,https://www.airbnb.es/rooms/583987826544046548...,Disfruta de la sencillez de este alojamiento t...,Quédate con Lina Maria,"Habitación en Barcelona, España",25 €,Horario de llegada: de 16:00 a 21:00,Salida antes de las 12:00,5 €,"Barcelona, Catalunya, España",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.airbnb.es/rooms/119285381097963570...,Sencillo en este lugar tranquilo y céntrico.Nú...,Anfitrión: Milan,"Habitación en Barcelona, España",85 €,Llegada a partir de las 15:00,Salida antes de las 11:00,10 €,"Barcelona, Catalunya, España",NaN,NaN,2,NaN,NaN
96,https://www.airbnb.es/rooms/2029507?adults=1&c...,Apartamento con 2 HABITACIONES y 2 BAÑOS COMPL...,Anfitrión: Vicky & Agus,"Alojamiento entero: apartamento en Barcelona, ...",130 €,Llegada a partir de las 14:00,Salida antes de las 11:00,110 €,NaN,NaN,NaN,4,4 viajeros · 2 dormitorios · 5 camas · 2 baños,NaN
97,https://www.airbnb.es/rooms/23661273?adults=1&...,Pequeño apartamento de 35 m2 recién reformado ...,Anfitrión: Anna,"Alojamiento entero: apartamento en Barcelona, ...",114 €,Llegada a partir de las 14:00,Salida antes de las 11:00,NaN,NaN,NaN,NaN,4,4 viajeros · 2 dormitorios · 2 camas · 1 baño,NaN
98,https://www.airbnb.es/rooms/5483016?adults=1&c...,"Este apartamento, situado en la Avenida Parale...",Anfitrión: Anders,"Alojamiento entero: apartamento en Barcelona, ...",96 €,Horario de llegada: de 14:00 a 21:00,Salida antes de las 10:00,60 €,NaN,NaN,NaN,4,4 viajeros · 2 dormitorios · 4 camas · 1 baño,NaN


In [4]:
# Para no correr todo el codigo, podemos basearnos en este csv y corregir lo que esta mal
df = pd.read_csv('df_testing_V2')
df

,Unnamed: 0,urls,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Cleaning_fees,Location,Ratings,Num_reviews,Maximum_guests,Beds,Baths,Rooms
0,0,https://www.airbnb.es/rooms/32571410?adults=1&...,"Habitación pequeña y acogedora, en ambiente ar...",Anfitrión: Ezequiel,Habitación privada en: apartamento en Barcelon...,28 €,Horario de llegada: de 15:00 a 22:00,Salida antes de las 10:00,18 €,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,1,https://www.airbnb.es/rooms/124782432007439961...,Disfruta de la sencillez de este alojamiento t...,Anfitrión: Kristina,"Habitación en Barcelona, España",36 €,Horario de llegada: de 16:00 a 20:00,Salida antes de las 11:00,31 €,"Barcelona, Catalunya, España",NaN,NaN,1.0,NaN,NaN,NaN
2,2,https://www.airbnb.es/rooms/48035540?adults=1&...,Este anfitrión tiene 200 evaluaciones de otros...,Anfitrión: Adrian,"Alojamiento entero: loft en Barcelona, España",81 €,Llegada a partir de las 16:00,Salida antes de las 12:00,25 €,"Barcelona, Catalunya, España",NaN,NaN,2.0,NaN,NaN,NaN
3,3,https://www.airbnb.es/rooms/125947582261568222...,"¡Te damos la bienvenida a tu hogar, lejos de c...",Anfitrión: Joonas,"Alojamiento entero: apartamento en Barcelona, ...",148 €,Máximo 4 huéspedes,No consta que el alojamiento tenga un detector...,50 €,"Barcelona, Catalunya, España",NaN,NaN,4.0,"4 viajeros · 2 dormitorios · 2 camas · 1,5 baños",NaN,NaN
4,4,https://www.airbnb.es/rooms/123992005587544231...,Estamos alquilando una PEQUEÑA HABITACIÓN PRIV...,Quédate con Jesus Maximiliano,"Habitación en Barcelona, España",61 €,Horario de llegada: de 15:00 a 20:00,Salida antes de las 10:00,NaN,"Barcelona, Catalunya, España",NaN,NaN,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,https://www.airbnb.es/rooms/893336277308710394...,Disfruta de la sencillez de este alojamiento t...,Quédate con Leo,"Habitación en Barcelona, España",45 €,Llegada a partir de las 15:00,Salida antes de las 11:00,15 €,NaN,NaN,NaN,2.0,NaN,NaN,NaN
96,96,https://www.airbnb.es/rooms/121306799954567354...,Este apartamento está situado en pleno centro ...,Quédate con Andrew,"Habitación en Barcelona, España",51 €,Horario de llegada: de 15:00 a 23:00,Salida antes de las 10:00,44 €,"Barcelona, Catalunya, España",NaN,NaN,1.0,NaN,NaN,NaN
97,97,https://www.airbnb.es/rooms/38512490?adults=1&...,Moderna y soleada Habitación en piso compart...,Quédate con Ammar,"Habitación en Barcelona, España",55 €,Llegada a partir de las 14:00,Salida antes de las 10:00,45 €,NaN,NaN,NaN,2.0,NaN,NaN,NaN
98,98,https://www.airbnb.es/rooms/127405422114360994...,Descubre Barcelona desde nuestro acogedor y to...,Anfitrión: Joonas,"Alojamiento entero: apartamento en Barcelona, ...",104 €,Máximo 2 huéspedes,No consta que el alojamiento tenga un detector...,30 €,"Barcelona, Catalunya, España",NaN,NaN,2.0,NaN,NaN,NaN
